In [38]:
import pickle
import pandas as pd
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

In [39]:
def Create_NN_Model(No_Features=100, No_Hidden_Layers=2, No_Hidden_Neurons=50, 
                    Hidden_Activation ="relu", No_OP_Neurons=1, 
                    Output_Activation="sigmoid", Kernel_Initializer="random_normal",
                    Optimizer="adam", Loss='binary_crossentropy', Metrics =['accuracy']):
  
    classifier = Sequential()

  ## Input Layer
    classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                       kernel_initializer=Kernel_Initializer, input_dim=No_Features))
  
  ## Hidden layers
    for i in range(No_Hidden_Layers):
        classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                         kernel_initializer=Kernel_Initializer))
    
  ## Output Layer
    classifier.add(Dense(No_OP_Neurons, activation=Output_Activation, 
                       kernel_initializer=Kernel_Initializer))
  
    classifier.compile(optimizer =Optimizer, loss=Loss, metrics = Metrics)

    return classifier

In [40]:
def Train_NN(NN_classifier, train_data, feature_list=[], Label_Col="Label", Batch_Size=50, Epochs=100):

    train_data.dropna()
    train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
    train_data[Label_Col] = pd.to_numeric(train_data[Label_Col], errors='coerce')
    train_data = train_data.dropna(subset=[Label_Col])
  
    train_features = train_data[feature_list]    
    train_labels = train_data[Label_Col]
    train_labels = train_labels.astype('int')

    NN_classifier.fit(train_features,train_labels, batch_size=Batch_Size, epochs=Epochs)

    eval_model=NN_classifier.evaluate(train_features, train_labels)
    print("Loss: ", eval_model[0])
    print("Accuracy of the model: ", eval_model[1])

    return NN_classifier

In [41]:
## Store trained model in a file to reuse in other codes without training again on same data

def Store_Trained_NN(NN_obj, Filepath):
  
    with open(Filepath, "wb") as file:
        pickle.dump(NN_obj, file)

In [42]:
## Load stored trained model and returns random forest model object

def Load_Trained_NN(Filepath):
  
    with open(Filepath, "rb") as file:
        NN_obj = pickle.load(file)

    return NN_obj

In [43]:
def Evaluate_NN(test_data, NN_Model_FilePath, feature_list=[], Label_Col="Label", threshold=0.5):
  
    test_data.dropna()
    test_data = pd.DataFrame(np.nan_to_num(np.array(test_data)),  columns = test_data.columns)
    test_data[Label_Col] = pd.to_numeric(test_data[Label_Col], errors='coerce')
    test_data = test_data.dropna(subset=[Label_Col])

    test_features = test_data[feature_list]
    test_labels = test_data[Label_Col]
    test_labels = test_labels.astype('int')

    NN_obj = Load_Trained_NN(NN_Model_FilePath) 
    predictions = NN_obj.predict(test_features)
    predictions_list = [int(p[0]) for p in predictions]
      
    true_subjective = 0
    true_objective = 0
    false_subjective = 0
    false_objective = 0
    
    for i in range(len(predictions_list)):
        if predictions_list[i] >= threshold:
            predictions_list[i] = 1
            if test_labels[i] == 1:
                true_subjective += 1
            else:
                false_subjective += 1
        else:
            predictions_list[i] = 0
            if test_labels[i] == 0:
                true_objective += 1
            else:
                false_objective += 1
      
    errors = abs(predictions_list - test_labels)

  # Calculate mean absolute error (MAE)
    MAE = round(np.mean(errors), 2)
  
  ## Confusion Matrix and Classification Report
    Confusion_Matrix = confusion_matrix(test_labels,predictions_list)
    Report = classification_report(test_labels,predictions_list)
    
    print("True Subjective : ", true_subjective)
    print("True Objective : ", true_objective)
    print("False Subjective : ", false_subjective)
    print("False Objective : ", false_objective)
    
    print("Accuracy: ", (true_subjective+true_subjective)/(true_subjective+true_subjective+false_subjective+false_objective)*100)
  
    return MAE, Confusion_Matrix, Report

In [44]:
Column_List = ["embedding"]
Label_Col = "label"
Vector_Size = 100
Embedding_Cols = ["emb"+str(i) for i in range(Vector_Size)]
# Data_Columns = Embedding_Cols
# Data_Columns = Data_Columns.append(Label_Col)
Column_List.append(Label_Col)

Folder_Path = "/Users/pranjali/Downloads/Wiki_BiasDetection/Data/Task1_FinalData/"

Train_Embedding_FilePath = Folder_Path + "Embeddings/train_emb.csv"
Test_Embedding_FilePath = Folder_Path + "Embeddings/test_emb.csv"
NN_Model_FilePath = "/Users/pranjali/Downloads/Wiki_BiasDetection/Baseline_Models/NN/NN_Train_Model.pkl"

train_data_raw = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data_raw = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)


In [45]:
def Get_Embeddings(data):
    
    Col_List = ["emb"+str(i) for i in range(100)]
    Col_List.append("label")
    
    Embeddings = []
    
    for i in range(data.shape[0]):
        row = data.iloc[i]
        embedding_str = row["embedding"][1:-1]
        embedding_list = embedding_str.split(',')
        embedding = [float(s) for s in embedding_list]
        embedding.append(int(row["label"]))
        Embeddings.append(embedding)
        
    return pd.DataFrame(Embeddings, columns=Col_List)

train_data = Get_Embeddings(train_data_raw)
test_data = Get_Embeddings(test_data_raw)

In [24]:
## Training Phase
NN_Classifier = Create_NN_Model()
NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols, Label_Col)
Store_Trained_NN(NN_obj, NN_Model_FilePath)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
13527/13527 [==============================] - 1s 46us/step - loss: 0.5084 - accuracy: 0.7415
Epoch 2/100
13527/13527 [==============================] - 0s 29us/step - loss: 0.3618 - accuracy: 0.8502
Epoch 3/100
13527/13527 [==============================] - 0s 29us/step - loss: 0.3507 - accuracy: 0.8547
Epoch 4/100
13527/13527 [==============================] - 0s 29us/step - loss: 0.3456 - accuracy: 0.8580
Epoch 5/100
13527/13527 [==============================] - 0s 28us/step - loss: 0.3420 - accuracy: 0.8578
Epoch 6/100
13527/13527 [==============================] - 0s 28us/step - loss: 0.3354 - accuracy: 0.8608
Epoch 7/100
13527/13527 [==============================] - 0s 29us/step - loss: 0.3281 - accuracy: 0.8635
Epoch 8/100
13527/13527 [==============================] - 0s 28us/step - loss: 0.3238 - accuracy: 0.8661
Epoch 9/100
13527/13527 [==============================] - 0s 28us/step - loss: 0.3158 - accuracy: 0.869

13527/13527 [==============================] - 0s 31us/step - loss: 0.1712 - accuracy: 0.9318
Epoch 75/100
13527/13527 [==============================] - 0s 31us/step - loss: 0.1766 - accuracy: 0.9291
Epoch 76/100
13527/13527 [==============================] - 0s 33us/step - loss: 0.1708 - accuracy: 0.9312
Epoch 77/100
13527/13527 [==============================] - 0s 31us/step - loss: 0.1668 - accuracy: 0.9338
Epoch 78/100
13527/13527 [==============================] - 0s 30us/step - loss: 0.1654 - accuracy: 0.9346
Epoch 79/100
13527/13527 [==============================] - 0s 32us/step - loss: 0.1676 - accuracy: 0.9328
Epoch 80/100
13527/13527 [==============================] - 0s 31us/step - loss: 0.1630 - accuracy: 0.9364
Epoch 81/100
13527/13527 [==============================] - 0s 30us/step - loss: 0.1607 - accuracy: 0.9365
Epoch 82/100
13527/13527 [==============================] - 0s 31us/step - loss: 0.1604 - accuracy: 0.9382
Epoch 83/100
13527/13527 [========================

In [37]:
MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, Label_Col, 0.5)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

KeyboardInterrupt: 

In [27]:
## Cross Validation

train_data.dropna()
train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
train_data[Label_Col] = pd.to_numeric(train_data[Label_Col], errors='coerce')
train_data = train_data.dropna(subset=[Label_Col])

train_features = train_data[Embedding_Cols]    
train_labels = train_data[Label_Col]
train_labels = train_labels.astype('int')

# create the sklearn model for the network
model_CV = KerasClassifier(build_fn=Create_NN_Model, verbose=1)

# we choose the initializers that came at the top in our previous cross-validation!!
kernel_initializer = ['random_uniform']
No_Hidden_Layers = [1, 2]
No_Hidden_Neurons= [10, 30, 50]
optimizer = ['adam', 'rmsprop']

batches = [64*x for x in range(1, 3)]
epochs = [50, 100, 150]

## We can also try different learning rates for optimizers. 
## For this create different objects of optimizers with different learning rates and pass list of objects

# grid search for initializer, batch size and number of epochs
param_grid = dict(epochs=epochs, batch_size=batches, Kernel_Initializer=kernel_initializer, 
                 No_Hidden_Layers=No_Hidden_Layers, Optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, cv=3, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(train_features, train_labels)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] Kernel_Initializer=random_uniform, No_Hidden_Layers=1, Optimizer=adam, batch_size=64, epochs=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
9018/9018 [==============================] - 0s 51us/step - loss: 0.6137 - accuracy: 0.6631
Epoch 2/50
9018/9018 [==============================] - 0s 24us/step - loss: 0.4100 - accuracy: 0.8306
Epoch 3/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3746 - accuracy: 0.8432
Epoch 4/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3614 - accuracy: 0.8496
Epoch 5/50
9018/9018 [==============================] - 0s 22us/step - loss: 0.3513 - accuracy: 0.8516
Epoch 6/50
9018/9018 [==============================] - 0s 22us/step - loss: 0.3449 - accuracy: 0.8542
Epoch 7/50
9018/9018 [==============================] - 0s 22us/step - loss: 0.3365 - accuracy: 0.8601
Epoch 8/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3338 - accuracy: 0.8586
Epoch 9/50
9018/9018 [==============================] - 0s 26us/step - loss: 0.3267 - accuracy: 0.8636
Epoch 10/50
9018/9018 [==============================] - 0s 27us/step - l

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.1s remaining:    0.0s


Epoch 1/50
9018/9018 [==============================] - 0s 43us/step - loss: 0.6267 - accuracy: 0.6454
Epoch 2/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.4109 - accuracy: 0.8368
Epoch 3/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3718 - accuracy: 0.8489
Epoch 4/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3577 - accuracy: 0.8552
Epoch 5/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3497 - accuracy: 0.8571
Epoch 6/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3416 - accuracy: 0.8621
Epoch 7/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3367 - accuracy: 0.8619
Epoch 8/50
9018/9018 [==============================] - 0s 22us/step - loss: 0.3330 - accuracy: 0.8643
Epoch 9/50
9018/9018 [==============================] - 0s 21us/step - loss: 0.3295 - accuracy: 0.8656
Epoch 10/50
9018/9018 [==============================] - 0s 21us/step - l

...

9018/9018 [==============================] - 0s 21us/step - loss: 0.2404 - accuracy: 0.9046
Epoch 84/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2417 - accuracy: 0.9050
Epoch 85/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2366 - accuracy: 0.9064
Epoch 86/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2376 - accuracy: 0.9072
Epoch 87/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2362 - accuracy: 0.9071
Epoch 88/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2342 - accuracy: 0.9076
Epoch 89/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2350 - accuracy: 0.9069
Epoch 90/150
9018/9018 [==============================] - 0s 22us/step - loss: 0.2309 - accuracy: 0.9088
Epoch 91/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2324 - accuracy: 0.9086
Epoch 92/150
9018/9018 [==============================] - 0s 21us/st

9018/9018 [==============================] - 0s 21us/step - loss: 0.3528 - accuracy: 0.8533
Epoch 8/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3492 - accuracy: 0.8542
Epoch 9/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3462 - accuracy: 0.8550
Epoch 10/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3448 - accuracy: 0.8572
Epoch 11/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3434 - accuracy: 0.8585
Epoch 12/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3377 - accuracy: 0.8592
Epoch 13/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3372 - accuracy: 0.8597
Epoch 14/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.3312 - accuracy: 0.8632
Epoch 15/150
9018/9018 [==============================] - 0s 22us/step - loss: 0.3310 - accuracy: 0.8637
Epoch 16/150
9018/9018 [==============================] - 0s 21us/step

9018/9018 [==============================] - 0s 21us/step - loss: 0.2206 - accuracy: 0.9120
Epoch 85/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2173 - accuracy: 0.9158
Epoch 86/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2182 - accuracy: 0.9145
Epoch 87/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2150 - accuracy: 0.9156
Epoch 88/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2152 - accuracy: 0.9144
Epoch 89/150
9018/9018 [==============================] - 0s 22us/step - loss: 0.2127 - accuracy: 0.9188
Epoch 90/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2097 - accuracy: 0.9196
Epoch 91/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2069 - accuracy: 0.9212
Epoch 92/150
9018/9018 [==============================] - 0s 21us/step - loss: 0.2064 - accuracy: 0.9216
Epoch 93/150
9018/9018 [==============================] - 0s 21us/st

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 30.5min finished


Epoch 1/50
13527/13527 [==============================] - 1s 93us/step - loss: 0.5749 - accuracy: 0.7109
Epoch 2/50
13527/13527 [==============================] - 1s 37us/step - loss: 0.3945 - accuracy: 0.8395
Epoch 3/50
13527/13527 [==============================] - 1s 44us/step - loss: 0.3659 - accuracy: 0.8501
Epoch 4/50
13527/13527 [==============================] - 1s 39us/step - loss: 0.3551 - accuracy: 0.8535
Epoch 5/50
13527/13527 [==============================] - 1s 42us/step - loss: 0.3475 - accuracy: 0.8570
Epoch 6/50
13527/13527 [==============================] - 1s 39us/step - loss: 0.3426 - accuracy: 0.8582
Epoch 7/50
13527/13527 [==============================] - 1s 39us/step - loss: 0.3367 - accuracy: 0.8604
Epoch 8/50
13527/13527 [==============================] - 1s 37us/step - loss: 0.3323 - accuracy: 0.8619
Epoch 9/50
13527/13527 [==============================] - 0s 37us/step - loss: 0.3273 - accuracy: 0.8650
Epoch 10/50
13527/13527 [==============================

In [28]:
# print results of cross validation

print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.8715901573499044 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'rmsprop', 'batch_size': 64, 'epochs': 50}
 mean=0.8715, std=0.001623 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 50}
 mean=0.8668, std=0.001206 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 100}
 mean=0.8574, std=0.002612 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 150}
 mean=0.8702, std=0.002748 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 50}
 mean=0.8697, std=0.003286 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 100}
 mean=0.8635, std=0.0006856 using {'Kernel_Initializer': 'random_uniform', 'No_Hid

In [30]:
# ## Train again using best parameter values identified by cross validation and store the trained model

# ## Training Phase
# NN_Classifier = Create_NN_Model(No_Hidden_Layers=grid_result.best_params_.No_Hidden_Layers, No_Hidden_Neurons=grid_result.best_params_.No_Hidden_Neurons, Kernel_Initializer=grid_result.best_params_.Kernel_Initializer, Optimizer=grid_result.best_params_.Optimizer)
# NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols, Label_Col, Batch_Size=grid_result.best_params_.batch_size, Epochs=grid_result.best_params_.epochs)
# Store_Trained_NN(NN_obj, NN_Model_FilePath)

## Store models trained with best parameters
Store_Trained_NN(grid, NN_Model_FilePath)

## Evaluation of above model on validation data
MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, Label_Col, 0.5)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)


3383/3383 [==============================] - 2s 488us/step
MEAN ABSOLUTE ERROR:  0.14


============ CONFUSION MATRIX ===============
[[1613  231]
 [ 252 1287]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1844
           1       0.85      0.84      0.84      1539

   micro avg       0.86      0.86      0.86      3383
   macro avg       0.86      0.86      0.86      3383
weighted avg       0.86      0.86      0.86      3383

Accuracy:  85.72273130357671
